# Deep ANN: Feature Engineering

In this notebook, we  will build a data pipeline using using [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx) to prepare features from the dataset:

* created an InteractiveContext to run TFX components interactively
* used TFX ExampleGen component to split your dataset into training and evaluation datasets
* generated the statistics and the schema of your dataset using TFX StatisticsGen and SchemaGen components
* validated the evaluation dataset statistics using TFX ExampleValidator
* performed feature engineering using the TFX Transform component


## Package Installation and Imports

For reference, the environment used *TFX Version: 1.10.0* *Tensorflow Version: 2.9.3* *Tensorflow-Transform Version: 1.10.1*

In [1]:
import os 

import tensorflow as tf

from tfx import v1 as tfx
import tensorflow_transform.beam as tft_beam
from google.protobuf.json_format import MessageToDict
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tensorflow_transform as tft

import tempfile
import pprint
import warnings

pp = pprint.PrettyPrinter()

# ignore tf warning messages
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore")

# Check the version
print('Tensorflow Extended Version: {}'.format(tfx.__version__))
print('Tensorflow Version: {}'.format(tf.__version__))
print('Tensorfow Transform Version: {}'.format(tft.__version__))

Tensorflow Extended Version: 1.11.0
Tensorflow Version: 2.10.1
Tensorfow Transform Version: 1.11.0


### Define paths

You will define a few global variables to indicate paths in the local workspace.

In [2]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'A_E_Fire_Dataset.csv')

### Preview the dataset

In [3]:
# Read datasets
import pandas as pd

df = pd.read_csv(_data_filepath, skipinitialspace=True)

# Preview the train set
df.head()

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1


### Create the Interactive Context

You will initialize the `InteractiveContext` below. This will create a database in the `_pipeline_root` directory which the different components will use to save or get the state of the component executions.

In [4]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
# You can safely ignore the warning about the missing config file.
context = InteractiveContext(pipeline_root=_pipeline_root)

## Run TFX components interactively

With that, you can now run the pipeline interactively. You will see how to do that as you go through the different components below.

### ExampleGen

You will start the pipeline with the [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component. This  will:

*   split the data into training and evaluation sets (by default: 2/3 train, 1/3 eval).
*   convert each data row into `tf.train.Example` format. This [protocol buffer](https://developers.google.com/protocol-buffers) is designed for Tensorflow operations and is used by the TFX components.
*   compress and save the data collection under the `_pipeline_root` directory for other components to access. These examples are stored in `TFRecord` format. This optimizes read and write operations within Tensorflow especially if you have a large collection of data.

Its constructor takes the path to your data source/directory. In our case, this is the `_data_root` path. The component supports several data sources such as CSV, tf.Record, and BigQuery. Since our data is a CSV file, we will use [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) to ingest the data.

Run the cell below to instantiate `CsvExampleGen`.

In [5]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

You can execute the component by calling the `run()` method of the `InteractiveContext`.

In [6]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 5
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

You will notice that an output cell showing the execution results is automatically shown. This metadata is recorded into the database created earlier. This allows you to keep track of your project runs. For example, if you run it again, you will notice the `.execution_id` incrementing.

The output of the components are called *artifacts* and you can see an example by navigating through  `.component.outputs > ['examples'] > Channel > ._artifacts > [0]` above. It shows information such as where the converted data is stored (`.uri`) and the splits generated (`.split_names`).

You can also examine the output artifacts programmatically with the code below.

In [7]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen\examples\5


As mentioned, the ingested data is stored in the directory shown in the `uri` field. It is also compressed using `gzip` and you can verify by running the cell below.

In [8]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

In a notebook environment, it may be useful to examine a few examples of the data especially if you're still experimenting. Since the data collection is saved in [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord), you will need to use methods that work with that data type. You will need to unpack the individual examples from the `TFRecord` file and format it for printing. 

In [9]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                     for name in os.listdir(train_uri)]

# Create a 'TFRecordDataset' to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [10]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the 'take()' method to specify how many record to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # covert the protocol buffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records


In [11]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'AIRFLOW': {'floatList': {'value': [0.0]}},
                           'DESIBEL': {'int64List': {'value': ['96']}},
                           'DISTANCE': {'int64List': {'value': ['10']}},
                           'FREQUENCY': {'int64List': {'value': ['75']}},
                           'FUEL': {'bytesList': {'value': ['Z2Fzb2xpbmU=']}},
                           'SIZE': {'int64List': {'value': ['1']}},
                           'STATUS': {'int64List': {'value': ['0']}}}}},
 {'features': {'feature': {'AIRFLOW': {'floatList': {'value': [0.0]}},
                           'DESIBEL': {'int64List': {'value': ['96']}},
                           'DISTANCE': {'int64List': {'value': ['10']}},
                           'FREQUENCY': {'int64List': {'value': ['72']}},
                           'FUEL': {'bytesList': {'value': ['Z2Fzb2xpbmU=']}},
                           'SIZE': {'int64List': {'value': ['1']}},
                           'STATUS': {'int64List': {'

## Data Validation

### StatisticsGen
The [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component computes statistics over your dataset for data analysis, as well as for use in downstream components (i.e. next steps in the pipeline). As mentioned earlier, this component uses TFDV under the hood so its output will be familiar to you.

`StatisticsGen` takes as input the dataset we just ingested using `CsvExampleGen`.

In [12]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the components
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [13]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

The [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) component also uses TFDV to generate a schema based on your data statistics.A schema defines the expected bounds, types, and properties of the features in your dataset.

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [14]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],)

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 7
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [15]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AIRFLOW',FLOAT,required,,-
'DESIBEL',INT,required,,-
'DISTANCE',INT,required,,-
'FREQUENCY',INT,required,,-
'FUEL',STRING,required,,'FUEL'
'SIZE',INT,required,,-
'STATUS',INT,required,,-


,Values
Domain,
'FUEL',"'gasoline', 'kerosene', 'lpg', 'thinner'"


### ExampleValidator

The [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component detects anomalies in your data based on the generated schema from the previous step. Like the previous two components, it also uses TFDV under the hood. 

`ExampleValidator` will take as input the statistics from `StatisticsGen` and the schema from `SchemaGen`. By default, it compares the statistics from the evaluation split to the schema from the training split.

In [16]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema = schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 8
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [17]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

With no anomalies detected, you can proceed to the next step in the pipeline.

### Transform
The [Transform](https://www.tensorflow.org/tfx/guide/transform) component performs feature engineering for both training and serving datasets. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library introduced in the first ungraded lab of this week.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module containing the preprocessing function.

In this section, you will work on an example of a user-defined Transform code. The pipeline needs to load this as a module so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few constants that group the data's attributes according to the transforms we will perform later. This file will also be saved locally.

In [18]:
# Set the constants module filename
_aefire_constants_module_file = 'aefire_constants.py'

In [19]:
%%writefile {_aefire_constants_module_file}

# Features to be scaled to the z-score
DENSE_FLOAT_FEATURE_KEYS = ['AIRFLOW']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['DISTANCE']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = {'DISTANCE': 3}

# Feature to scale from 0 to 1
RANGE_FEATURE_KEYS = ['DESIBEL', 'FREQUENCY', 'SIZE']

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 10

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 5

# Features with string data types that will be converted to indices
VOCAB_FEATURE_KEYS = ['FUEL']

# Feature that the model will predict
STATUS_KEY = 'STATUS'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting aefire_constants.py


Next, you will work on the module that contains `preprocessing_fn()`. This function defines how you will transform the raw data into features that your model can train on (i.e. the next step in the pipeline). You will use the [tft module functions](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) to make these transformations.

In [20]:
# Set the transform module filename
_aefire_transform_module_file = 'aefire_transform.py'

In [21]:
%%writefile {_aefire_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import aefire_constants

# Unpack the contents of the constants module
_DENSE_FLOAT_FEATURE_KEYS = aefire_constants.DENSE_FLOAT_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = aefire_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = aefire_constants.FEATURE_BUCKET_COUNT
_RANGE_FEATURE_KEYS = aefire_constants.RANGE_FEATURE_KEYS
_VOCAB_SIZE = aefire_constants.VOCAB_SIZE
_OOV_SIZE = aefire_constants.OOV_SIZE
_VOCAB_FEATURE_KEYS = aefire_constants.VOCAB_FEATURE_KEYS
_STATUS_KEY = aefire_constants.STATUS_KEY
_transformed_name = aefire_constants.transformed_name

# Define the transformantions
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
    inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
    Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the z-score
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])

    # Bucketize the feature
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Scale these feature in range [0,1]
    for key in _RANGE_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])

    # Transform the strings into indices
    for key in _VOCAB_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            inputs[key],
            top_k=(_VOCAB_SIZE),
            num_oov_buckets=(_OOV_SIZE))
    
    # Keep the features as is. No tft function needed
    outputs[_transformed_name(_STATUS_KEY)] = inputs[_STATUS_KEY]
    
    return outputs

Overwriting aefire_transform.py


You can now pass the training data, schema, and transform module to the `Transform` component. You can ignore the warning messages generated by Apache Beam regarding type hints.

In [22]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_aefire_transform_module_file))

# Run the component.
# The `enable_cache` flag is disabled in case you need to update your transform module file.
context.run(transform, enable_cache=False)

RuntimeError: Failed to build wheel.